In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import matplotlib.pyplot as plt
import numpy as np

def gen_matrix(image, nb_channels, random_state):
      matrix_A = np.array([[0, -1, 0],
                           [-1, 4, -1],
                           [0, -1, 0]])
      matrix_B = np.array([[0, 0, 0],
                          [0, -4, 1],
                          [0, 2, 1]])
      if random_state.rand() < 0.5:
          return [matrix_A] * nb_channels
      else:
          return [matrix_B] * nb_channels
# aug = iaa.Convolve(matrix=gen_matrix)

seq = iaa.Sequential([
    ## unsure if Crop is likely to be useful offsett may be better
#     iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    ## definitely worth doing Fliplr as it doubles the size.
#     iaa.Fliplr(0.5), # horizontally flip 50% of the images
    ## GaussianBlur is also worth performing on for generalising
    ## possibly ignore it for my own data?  
#     iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    ## Noise is also worth doing for generalising.
#     iaa.AdditiveGaussianNoise(scale=0.1*255) #looks like static
#     iaa.EdgeDetect(alpha=(0.0, 1.0))
    
    ## I think convolve is probably useful if you define it with the sharpness to demonstrate
    ## the advantages of using Convolve is to view what the Computer might see.
    iaa.Convolve(matrix=gen_matrix),
    ## sharpeness definitely feels useful for defining the edges of the area.
#     iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0))
    ## probably worth doing sometimes, it might be better with 
#     iaa.ContrastNormalization((0.5, 1.5))
#     iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ## Affine scale
    ## might be an issue due to where the signs are typically located don't increase the sign beyond 1.0
    ## unless you write a function to discard bounding boxes outside the actual area.
#     iaa.Affine(scale=(0.35, .90)),
    ## scale indepently on each axis indepently changing axis, might be really useful
#     iaa.Affine(scale={'x': (0.5, 1), 'y': (0.35, 1.0)}),
    ## this could be useful however, it would require it the translation to not cut off the signs in the images
    ## possible function to ensure it's not missing
    ## note though it's on percent
#     iaa.Affine(translate_percent={'x': (-0.02, 0.2), 'y': (-0.2, 0.2)})
    ## note the below is a pixel translation which might actually be better due to it being more minor
#     iaa.Affine(translate_pixel={'x': (-20, 20), 'y': (-20, 20)})
    ## note that this will be worth while if it's using with scaling and/or doesn't cut off
    ## the actual sign
#     iaa.Affine(rotate=(-45, 45))
    ## shear is definitely worth including in the data augmentation process
#     iaa.Affine(shear=(-160,160))
    ## unsure of what the mode kwarg is doing currently
#     iaa.Affine(translate_percent={"x": (-0.20)}, mode=ia.ALL, cval=(0, 255))
    ## Really useful on low level distortion
#     iaa.PiecewiseAffine(scale=(0.008, 0.0095))
    ## use ElasticTransformation to add random can pass in tuples for randomness
    ## to the distortion field
    iaa.ElasticTransformation(alpha=(0, 5.0), sigma=(0.25, 1))

])
img1 = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_235.jpg')

img = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_105.jpg')
# cv2.imshow('test', img)
imgs = [img, img1]
print(type(img))
# seq.show_grid(imgs[0], cols=8, rows=8)
# cropped = img[7:100, 22:67]
# # cv2.imshow('cropped', cropped)
# # cv2.waitKey(0)

img_aug = seq.augment_image(img)
# seq.show_grid(img_aug, cols=8, rows=8)
# print(img.shape)
# print(img_aug.shape)
# print(len(img_aug))

# cv2.imshow('aug??', img_aug)
# cv2.waitKey(0)

# # for batch_idx in range(1000):
# #     # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
# #     # or a list of 3D numpy arrays, each having shape (height, width, channels).
# #     # Grayscale images must have shape (height, width, 1) each.
# #     # All images must have numpy's dtype uint8. Values are expected to be in
# #     # range 0-255.
# #     images = load_batch(batch_idx)
# #     images_aug = seq.augment_images(images)
# #     train_on_images(images_aug)

<class 'numpy.ndarray'>


# Write out one image as new.

1. It needs the path of the image.
2. The bounding boxes of the image.
3. Write the number 

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2

base_dir = '/Users/datascience4/Documents/training3'
print(len(os.listdir(base_dir)))
xml_files = os.listdir(base_dir)
xml_files = [x for x in os.listdir(base_dir) if '.xml' in x]
print(xml_files)
items = []

## come back to this

#let's read the first xml
test_xml = os.path.join(base_dir, xml_files[0])
tree = ET.parse(test_xml)
root = tree.getroot()
## path can be used for cv2 read image
path = root.find('path').text
print(path)
## find the object and xmin, ymin, xmax, ymax
for child in root.iter('object'):
    if child.find('bndbox'):
        bndbox = child.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
print(xmin, ymin, xmax, ymax)

# ia.seed(1)
## pass the path into cv2
image = cv2.imread(path)
# cv2.imshow('path', image)
# cv2.waitKey(0)

## bounding boxes now for the image takes a list
bbs = ia.BoundingBoxesOnImage([
    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect bb
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
#         scale=(0.5, 0.7),
        rotate=(-25, 25)
        
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

seq = iaa.Sequential([
    iaa.Fliplr(0.5), #horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    #Small gaussian blur with random sigma between 0 and 0.5
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
                 iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    #Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    #Add gaussian noise.
    #For 50% of all images, we sample the noise once per pixel..
    #For the other 50% of all images, we sample the noise per pixel
    #AND channel. This can change the color (not only the brightness)
    #of the pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255),
                             per_channel=0.5),
    #Make some images brighter and some darker.
    #in 20% of all cases we sample the multiplier once per channel.
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    #Apply affine transformations to each image.
    #Scale/zoom them, translate/move them, rotate them, and shear them.
    iaa.Affine(
        scale={'x': (0.8, 1.2), 'y': (0.8, 1.2)},
        translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) #apply augmenters in random order

##This sequence is getting played first now
## Transformation have to be played out this order.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), #horizontal flips
    iaa.Crop(percent=(0,0.1)), #random crops
    #gaussian blur is removed
    #contrast is removed
    #additive gaussian noise removed
    #multiply removed
    iaa.Affine(
        scale={'x': (0.8, 1.2), 'y': (0.8, 1.2)},
        translate_percent={'x': (-0.2, 0.2), 'y':(-0.2,0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True)

color_seq = iaa.Sequential([
    iaa.Sometimes(0.5,
                 iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255),
                             per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2)
], random_order=True)

color_seq = iaa.Sequential([
    #Small gaussian blur with random sigma between 0 and 0.5
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
                 iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    #Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    #Add gaussian noise.
    #For 50% of all images, we sample the noise once per pixel..
    #For the other 50% of all images, we sample the noise per pixel
    #AND channel. This can change the color (not only the brightness)
    #of the pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255),
                             per_channel=0.5),
    #Make some images brighter and some darker.
    #in 20% of all cases we sample the multiplier once per channel.
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
], random_order=True) #apply augmenters in random order

## make our sequences deterministic.
# we can now apply it to the image and then to the bbs and it will
# lead to the same augmentations
# IMPORTANT: call this once PER BATCH, otherwise you will always
# get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()
color_seq_det = color_seq.to_deterministic()
##Augment bbs and images.
## as we only have one image and list of bbs we use
## [image] and [bbs] to turn both into lists (batches) for the
## functions and then [0] to reverse that. In a real experiment, your
## variables would likely already be lists.
#image_aug does the 'moving' augmentations on  the boxes.
image_aug = seq_det.augment_images([image])[0]
#color_aug does the color related augmentations this won't be used
#until later
color_aug = color_seq_det.augment_images([image])[0]
#transformations on the bounding boxes
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

##print the coordinates before/after augmentation(see below)
## use .x1_int, y1_int, ... to get interger coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print(f"BB {i}:, ({before.x1, before.y1, before.x2, before.y2} \
          -> ({after.x1, after.y1, after.x2, after.y2})")
    
# image with bbs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=2)
# cv2.imshow('image before', image_before)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[0, 0, 255])
# cv2.imshow('image after', image_after)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imwrite('/Users/datascience4/Desktop/test1.png', image_after)

385
['A3400_NB1_RAV_R07_180516084515_frame_3598.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2862.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2447.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2335.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4022.xml', 'A3400_NB1_RAV_R07_180516084515_frame_224.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2255.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2527.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3163.xml', 'A3400_NB1_RAV_R07_180516084515_frame_350.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3375.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3374.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3837.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3162.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2240.xml', 'A3400_NB1_RAV_R07_180516084515_frame_225.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4023.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2446.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2863.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3599.xml', 'A3400_NB1_RAV_R07

True

## Things to do
1. Create a folder for the augmented images
2. Then run through all images (drawing on each image the bounding box)
3. Sense check that all are in the bounding boxes.

In [3]:
try:
    os.mkdir('/Users/datascience4/Documents/training_3_data_aug')
except FileExistsError:
    print('folder already exists')

folder already exists


In [4]:
aug_dir = '/Users/datascience4/Documents/training_3_data_aug'

## SECTION DEALS WITH BOUNDING BOXES NOT IN THE IMAGE
## funtion to pad the image and deal with bounding boxes not in the images

# Pad image with a 1px white and (BY-1)px black border
def pad(image, by):
    if by <= 0:
        return image
    image_border1 = np.pad(
        image, ((1, 1), (1, 1), (0, 0)),
        mode="constant", constant_values=255
    )
    image_border2 = np.pad(
        image_border1, ((by-1, by-1), (by-1, by-1), (0, 0)),
        mode="constant", constant_values=0
    )
    return image_border2

# Draw BBs on an image
# and before doing that, extend the image plane by BORDER pixels.
# Mark BBs inside the image plane with green color, those partially inside
# with orange and those fully outside with red.
def draw_bbs(image, bbs, border):
    ##rgb in open cv isn't rgb but bgr - it's bad for that!
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
        elif bb.is_partly_within_image(image.shape):
            print('partly?')
            color = ORANGE
        else:
            print('not in at all')
            color = RED
        image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border

# def function 

def draw_bbs_count(image, bbs, border, **kwargs):
    counter = kwargs.get('counter', None)
    ## using open CV2 so colours are in BGR and not RGB so check the order if it comes different
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    ## probably can remove the ine beloe as it won't be needed
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
            counter += 1
        elif bb.is_partly_within_image(image.shape):
            print('partly version 2\n\n\n')
            color = ORANGE
            image_border = None
        else:
            print('not in at all\n\n\n')
            color = RED
            image_border = None
        if image_border is not None:
            image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border, counter
    


## write out a file that makes ue of similar layout as draw_bbs



## attempt to write out the same image multiple times in different ways
for i in range(10):
    #let's read the first xml
    test_xml = os.path.join(base_dir, xml_files[i])
    print(i)
    tree = ET.parse(test_xml)
    root = tree.getroot()
    ## path can be used for cv2 read image
    path = root.find('path').text
    print(path)
    ## find the object and xmin, ymin, xmax, ymax
    for child in root.iter('object'):
        if child.find('bndbox'):
            bndbox = child.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
    
    image = cv2.imread(path)
    bbs = ia.BoundingBoxesOnImage([
                    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
    ], shape=image.shape)
    ## new
    counter = 0
    # counter for partials
    partial_counter = 0
    # counter for non black identified images
    non_zero_counter = 0
    ## Set the counter to less than the amount of images you actually want, so if it's 50 of each individual image
    while counter < 50:
        print('image number: ', counter + 1)
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
#         print('bbs_aug', bbs_aug.bounding_boxes)
#         print('type', type(bbs_aug.bounding_boxes))
        ## take the bbs_aug, and . then measure it on the image_aug if the image is fully black it's wrong
        bbs_aug_array = np.array([
            bbs_aug.bounding_boxes[0].y1,
            bbs_aug.bounding_boxes[0].y2,\
            bbs_aug.bounding_boxes[0].x1,\
            bbs_aug.bounding_boxes[0].x2])
        bbs_aug_array = bbs_aug_array.astype(int)
        image_aug_cropped = image_aug[bbs_aug_array[0]:bbs_aug_array[1], bbs_aug_array[2]:bbs_aug_array[3]]
#         is_black = np.array([0,0,0], dtype='uint8')
#         is_quite_black = np.array([20, 20, 20], dtype='uint8')
#         mask = cv2.inRange(image_aug_cropped, is_black, is_quite_black)
#         output = cv2.bitwise_and(image_aug_cropped, image_aug_cropped, mask = mask)
#         output1 = output.copy()
#         output1[mask == 0] = (255, 255, 255)
        ## convert image_aug_cropped to grayscale
        image_aug_cropped_grayscale = cv2.cvtColor(image_aug_cropped, cv2.COLOR_BGR2GRAY)
        if image_aug_cropped_grayscale is not None:
            print('shape of the grayscale cropped image', image_aug_cropped_grayscale.shape)
            image_aug_cropped_grayscale_array = np.array(list(image_aug_cropped_grayscale.shape))
            print(image_aug_cropped_grayscale_array[0] * image_aug_cropped_grayscale_array[1] / 100 * 40)
            if cv2.countNonZero(image_aug_cropped_grayscale) \
            < (image_aug_cropped_grayscale_array[0] \
            * image_aug_cropped_grayscale_array[1] / 100 * 70):
                print("Image is black")
    #             cv2.imshow('crop1', image_aug_cropped_grayscale)
    #             cv2.waitKey(0)
                partial_counter += 1
            else:
                non_zero_counter += 1
                print('Colored image')
#                 print('bbs_aug', bbs_aug)
                #counter is adding here.
                image_aug = color_seq_det.augment_images([image_aug])[0]
                image_after, counter = draw_bbs_count(image_aug, bbs_aug, 1, counter=counter)
                
                if image_after is not None:
                    cv2.imwrite(os.path.join(aug_dir, ('bb_count_photo' + str(i) + 'counter_no'\
                                                + str(counter) + '.jpeg')), image_after)

#         is_black = np.array([0,0,0], dtype="uint8")
#             print(type(is_black))
#             bb_array = np.array([bb.y1, bb.y2, bb.x1, bb.x2])
#             bb_array = bb_array.astype(int)
#             print(bb_array)
# #             mask = cv2.inRange(image, is_black, is_black)
# #             output = cv2.bitwise_and(image, image, mask = mask)
# #             # show the images
# #             cv2.imshow("images", np.hstack([image, output]))
# #             cv2.waitKey(0)
#             cropped = image_border[bb_array[0]: bb_array[1], bb_array[2], bb_array[3]]
#             cv2.imshow('crop 1', cropped)
#             cv2.waitKey(0)
        
        
    ## I don't think this is required now
#     for j in range(10):
#         seq_det = seq.to_deterministic()
#         image_aug = seq_det.augment_images([image])[0]
#         bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
#         ## test removing
#         image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[255, 140, 0])
#         ## testing
#         image_after1 = draw_bbs(image_aug, bbs_aug, 100, j)
#         cv2.imwrite(os.path.join(aug_dir, ('bb' + str(i) + str(j) + '.png')), image_after1)
# #         image_after2 = draw_bbs(image_aug, bbs_aug.remove_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed' + str(i) + str(j) + '.png')), image_after2)
# #         image_after3 = draw_bbs(image_aug, bbs_aug.remove_out_of_image()\
# #                 .cut_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed_and_cut_out' + str(i) + str(j) + '.png')), image_after3)
# #         cv2.imwrite(os.path.join(aug_dir, (str(i) + str(j) + '.png')), image_after)
#         print(bbs, bbs_aug)


0
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_3598.jpg
image number:  1
shape of the grayscale cropped image (54, 74)
1598.4
Colored image
image number:  2
shape of the grayscale cropped image (59, 67)
1581.2
Colored image
image number:  3
shape of the grayscale cropped image (74, 63)
1864.8
Colored image
image number:  4
shape of the grayscale cropped image (50, 68)
1360.0
Colored image
image number:  5
shape of the grayscale cropped image (56, 73)
1635.2
Colored image
image number:  6
shape of the grayscale cropped image (67, 91)
2438.8
Colored image
image number:  7
shape of the grayscale cropped image (67, 75)
2010.0
Colored image
image number:  8
shape of the grayscale cropped image (80, 75)
2400.0
Colored image
image number:  9
shape of the grayscale cropped image (62, 56)
1388.8
Colored image
image number:  10
shape of the grayscale cropped image (72, 81)
2332.8
Colored image
image number:  11
shape of the grayscale cropped imag

image number:  43
shape of the grayscale cropped image (41, 43)
705.1999999999999
Colored image
image number:  44
shape of the grayscale cropped image (27, 34)
367.2
Colored image
image number:  45
image number:  45
shape of the grayscale cropped image (29, 37)
429.20000000000005
Colored image
image number:  46
shape of the grayscale cropped image (28, 35)
392.0
Colored image
image number:  47
shape of the grayscale cropped image (38, 35)
532.0
Colored image
image number:  48
shape of the grayscale cropped image (33, 38)
501.59999999999997
Colored image
image number:  49
shape of the grayscale cropped image (27, 46)
496.8
Colored image
image number:  50
shape of the grayscale cropped image (40, 31)
496.0
Colored image
2
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_2447.jpg
image number:  1
shape of the grayscale cropped image (83, 60)
1992.0
Colored image
image number:  2
shape of the grayscale cropped image (64, 47)
1203.1999999999998


not in at all



image number:  24
shape of the grayscale cropped image (46, 63)
1159.2
Image is black
image number:  24
shape of the grayscale cropped image (46, 60)
1104.0
Colored image
image number:  25
shape of the grayscale cropped image (53, 60)
1272.0
Colored image
image number:  26
image number:  26
shape of the grayscale cropped image (47, 62)
1165.6
Colored image
image number:  27
shape of the grayscale cropped image (48, 64)
1228.8
Colored image
image number:  28
image number:  28
shape of the grayscale cropped image (46, 55)
1012.0
Colored image
image number:  29
shape of the grayscale cropped image (53, 63)
1335.6
Colored image
image number:  30
shape of the grayscale cropped image (55, 53)
1166.0
Image is black
image number:  30
image number:  30
shape of the grayscale cropped image (48, 47)
902.4
Image is black
image number:  30
shape of the grayscale cropped image (53, 48)
1017.6
Colored image
image number:  31
shape of the grayscale cropped image (32, 60)
768.0
Colored

image number:  44
shape of the grayscale cropped image (119, 250)
11900.0
Colored image
image number:  45
image number:  45
shape of the grayscale cropped image (163, 322)
20994.4
Colored image
image number:  46
shape of the grayscale cropped image (148, 273)
16161.6
Colored image
image number:  47
shape of the grayscale cropped image (128, 230)
11776.0
Colored image
image number:  48
image number:  48
image number:  48
image number:  48
image number:  48
shape of the grayscale cropped image (170, 250)
17000.0
Colored image
image number:  49
shape of the grayscale cropped image (133, 266)
14151.199999999999
Colored image
image number:  50
shape of the grayscale cropped image (178, 223)
15877.6
Colored image
5
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_224.jpg
image number:  1
image number:  1
image number:  1
image number:  1
shape of the grayscale cropped image (95, 46)
1748.0
Colored image
image number:  2
shape of the grayscale cro

image number:  49
shape of the grayscale cropped image (113, 69)
3118.8
Colored image
not in at all



image number:  49
shape of the grayscale cropped image (84, 59)
1982.4
Colored image
image number:  50
shape of the grayscale cropped image (73, 46)
1343.1999999999998
Image is black
image number:  50
shape of the grayscale cropped image (112, 72)
3225.6
Colored image
6
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_2255.jpg
image number:  1
shape of the grayscale cropped image (69, 50)
1380.0
Colored image
not in at all



image number:  1
image number:  1
shape of the grayscale cropped image (68, 53)
1441.6
Colored image
image number:  2
shape of the grayscale cropped image (52, 55)
1144.0
Colored image
image number:  3
shape of the grayscale cropped image (54, 51)
1101.6
Colored image
image number:  4
shape of the grayscale cropped image (53, 60)
1272.0
Colored image
image number:  5
shape of the grayscale cropped image (58, 68)
1577.

not in at all



image number:  9
shape of the grayscale cropped image (46, 42)
772.8
Colored image
image number:  10
shape of the grayscale cropped image (34, 38)
516.8
Image is black
image number:  10
shape of the grayscale cropped image (40, 49)
784.0
Colored image
not in at all



image number:  10
shape of the grayscale cropped image (39, 36)
561.5999999999999
Image is black
image number:  10
shape of the grayscale cropped image (35, 31)
434.0
Colored image
image number:  11
shape of the grayscale cropped image (38, 38)
577.6
Colored image
image number:  12
shape of the grayscale cropped image (38, 42)
638.4000000000001
Colored image
image number:  13
shape of the grayscale cropped image (35, 41)
574.0
Colored image
image number:  14
image number:  14
shape of the grayscale cropped image (29, 30)
348.0
Colored image
image number:  15
shape of the grayscale cropped image (35, 41)
574.0
Colored image
image number:  16
shape of the grayscale cropped image (37, 38)
562.4
Colored image

image number:  30
shape of the grayscale cropped image (83, 85)
2822.0
Colored image
image number:  31
shape of the grayscale cropped image (93, 89)
3310.7999999999997
Colored image
image number:  32
shape of the grayscale cropped image (91, 98)
3567.2000000000003
Colored image
image number:  33
image number:  33
shape of the grayscale cropped image (92, 86)
3164.8
Colored image
image number:  34
shape of the grayscale cropped image (92, 95)
3496.0
Colored image
image number:  35
shape of the grayscale cropped image (83, 77)
2556.3999999999996
Colored image
image number:  36
shape of the grayscale cropped image (83, 104)
3452.7999999999997
Colored image
image number:  37
image number:  37
shape of the grayscale cropped image (86, 102)
3508.8
Colored image
image number:  38
shape of the grayscale cropped image (77, 63)
1940.3999999999999
Colored image
image number:  39
shape of the grayscale cropped image (90, 98)
3528.0
Colored image
image number:  40
image number:  40
shape of the gra

not in at all



image number:  17
shape of the grayscale cropped image (77, 80)
2464.0
Colored image
image number:  18
shape of the grayscale cropped image (94, 73)
2744.8
Colored image
image number:  19
shape of the grayscale cropped image (100, 89)
3560.0
Colored image
not in at all



image number:  19
shape of the grayscale cropped image (75, 78)
2340.0
Image is black
image number:  19
shape of the grayscale cropped image (108, 82)
3542.4
Colored image
image number:  20
image number:  20
image number:  20
shape of the grayscale cropped image (82, 65)
2132.0
Image is black
image number:  20
shape of the grayscale cropped image (73, 62)
1810.3999999999999
Image is black
image number:  20
shape of the grayscale cropped image (74, 81)
2397.6
Image is black
image number:  20
image number:  20
image number:  20
image number:  20
image number:  20
shape of the grayscale cropped image (69, 84)
2318.4
Colored image
not in at all



image number:  20
shape of the grayscale cropped image (76

image number:  46
shape of the grayscale cropped image (99, 85)
3366.0
Colored image
not in at all



image number:  46
shape of the grayscale cropped image (80, 60)
1920.0
Image is black
image number:  46
image number:  46
shape of the grayscale cropped image (89, 64)
2278.4
Image is black
image number:  46
image number:  46
shape of the grayscale cropped image (94, 65)
2444.0
Image is black
image number:  46
image number:  46
shape of the grayscale cropped image (83, 78)
2589.6
Colored image
not in at all



image number:  46
image number:  46
shape of the grayscale cropped image (94, 81)
3045.6
Colored image
not in at all



image number:  46
image number:  46
shape of the grayscale cropped image (74, 63)
1864.8
Image is black
image number:  46
image number:  46
shape of the grayscale cropped image (84, 82)
2755.2
Colored image
not in at all



image number:  46
shape of the grayscale cropped image (77, 66)
2032.8
Colored image
not in at all



image number:  46
shape of the graysca

In [5]:
print('partial counter: ', partial_counter)
print('non-zero counter as in colour exists greater than 10% of all pixels', non_zero_counter)

partial counter:  77
non-zero counter as in colour exists greater than 10% of all pixels 104


# The above works but needs improvement
Things now required.
* ~~Bounding boxes are now getting detected inside each image, as well as in the augmented images whether or not they exist in boundaries of that image using the function draw_bbs with padding.~~
* ~~Even on minor partial images we now need to cut them and redo the augmentations.~~
* Now need to add the colour transformations.
* Write a function that only write an xml file and bounding box coordinates if the are not just partially in but fully in.
* If partial or out the augmentation needs to go again until randomly generating one with bounding boxes inside the image. Otherwise it will be hinder the model.
* A function similar to draw_bbs should be able to accomplish this.

* Bounding boxes need to be in the image, if they aren't they should be disgarded and a new image created that until the bounding box sits inside the image and can then be counted.


* Address the category imbalance as well with this work. That if takes the classes, and counts them, that it should then run each for the x amount of classes. related to each type of work.

In [6]:
print(bbs.bounding_boxes)
for bb in bbs.bounding_boxes:
    print(bb)

[BoundingBox(x1=1260.0000, y1=106.0000, x2=1317.0000, y2=172.0000, label=None)]
BoundingBox(x1=1260.0000, y1=106.0000, x2=1317.0000, y2=172.0000, label=None)


In [7]:
image.shape


(768, 1360, 3)

In [8]:
import cv2
import numpy as np
test = cv2.imread('/Users/datascience4/Documents/training_3_data_aug/bb_count_photo9counter_no14.jpeg')

is_black = np.array([0,0,0], dtype="uint8")
is_quite_black = np.array([20, 20, 20], dtype='uint8')
mask = cv2.inRange(test, is_black, is_quite_black)
output = cv2.bitwise_and(test, test, mask = mask)
output1 = output.copy()
output1[mask == 0] = (255, 255, 255)
cv2.imshow('output1', output1)
cv2.waitKey(0)

48